In [ ]:
from openai import AzureOpenAI
import pandas as pd
import re
import time
import csv

In [ ]:
client = AzureOpenAI(
    azure_endpoint = " ",
    api_key="<your_api_key>",
    api_version=" "
) 

In [ ]:
# Divide yoru dataframe into chucks if it is large
'''
num_chunks = 10
chunk_size = len(df) // num_chunks

# Split the DataFrame into 10 chunks
chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

# Assign each chunk to a separate variable df1 to df10
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10 = chunks
'''

In [ ]:
# CAMS (can be adapted to binary and multi-class classification datasets)
df = pd.read_csv("CAMS_var.csv")
system_prompt = "You are a mental health specialist. Analyze the provided text for signs of mental health issues like depression, anxiety, or other mental disorders. Take social factors such as race, gender, age, sexuality, nationality, and religion into account. Provide a brief explanation of your prediction considering both social factors and the text given."
predictions = []
reasonings = []
bad_sample = []

for i, row in df.iterrows():
    user_prompt = f"Given the post: '{row['text']}', analyze the content to identify the most possible cause of the user's mental issue. The output should be in the form of 'OUTPUT: 0 (No reason)' or 'OUTPUT: 1 (Bias or abuse)' or 'OUTPUT: 2 (Jobs and Careers)' or 'OUTPUT: 3 (Medication)' or 'OUTPUT: 4 (Relationship)' or 'OUTPUT: 5 (Alienation)'; REASONING:"
    try:
        response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ])
        content = response.choices[0].message.content      
        time.sleep(0.1)
        prediction_match = re.search(r'OUTPUT: (\d+)', content)
        reasoning_match = re.search(r'REASONING: (.*)', content)
        prediction = prediction_match.group(1) if prediction_match else None
        reasoning = reasoning_match.group(1) if reasoning_match else None
        if i % 50 == 0:
            print('Progress:', i, 'finished')
    except Exception as e:
        prediction = None
        reasoning = None
        bad_sample.append((i, str(e)))
        print('Error:', i, str(e)[:8])
    predictions.append(prediction)
    reasonings.append(reasoning)


df['prediction'] = predictions
df['reasoning'] = reasonings
df.to_csv("CAMS_var_GPT4.csv", index=False)
csv_file = "CAMS_var_GPT4_bad_sample_index.csv"

# Writing the list of tuples to a CSV file
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Writing the header (if necessary)
    writer.writerow(["Row_num", "Error_msg"])
    
    # Writing the data
    writer.writerows(bad_sample)

print(f"Data saved to {csv_file}")


In [ ]:
# IRF
'''
df = pd.read_csv("IRF_var.csv")
system_prompt = "You are a mental health specialist. Analyze the provided text for signs of mental health issues like depression, anxiety, or other mental disorders. Take social factors such as race, gender, age, sexuality, nationality, and religion into account. Provide a brief explanation of your prediction considering both social factors and the text given."
tb_predictions = []
tb_reasonings = []
pb_predictions = []
pb_reasonings = []
bad_sample = []

for i, row in df.iterrows():
    user_prompt = f"Given the post: '{row['text']}', analyze the content to identify the presence of Thwarted Belongingness and Perceived Burdensomeness. The output should be in the following format:\n\n'Thwarted Belongingness: OUTPUT: 0 (No) or 1 (Yes); REASONING:'\n\n'Perceived Burdensomeness: OUTPUT: 0 (No) or 1 (Yes); REASONING:"
    try:
        response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ])
        content = response.choices[0].message.content      
        time.sleep(0.1)
        
        # Extract Thwarted Belongingness
        tb_prediction_match = re.search(r'Thwarted Belongingness: OUTPUT: (\d+)', content)
        tb_reasoning_match = re.search(r'Thwarted Belongingness.*?REASONING: (.*?)(?=Perceived Burdensomeness|$)', content, re.DOTALL)
        tb_prediction = tb_prediction_match.group(1) if tb_prediction_match else None
        tb_reasoning = tb_reasoning_match.group(1).strip() if tb_reasoning_match else None
        
        # Extract Perceived Burdensomeness
        pb_prediction_match = re.search(r'Perceived Burdensomeness: OUTPUT: (\d+)', content)
        pb_reasoning_match = re.search(r'Perceived Burdensomeness.*?REASONING: (.*?)(?=Thwarted Belongingness|$)', content, re.DOTALL)
        pb_prediction = pb_prediction_match.group(1) if pb_prediction_match else None
        pb_reasoning = pb_reasoning_match.group(1).strip() if pb_reasoning_match else None
        if i % 50 == 0:
            print('Progress:', i, 'finished')
    except Exception as e:
        tb_prediction = None
        tb_reasoning = None
        pb_prediction = None
        pb_reasoning = None
        bad_sample.append((i, str(e)))
        print('Error:', i, str(e)[:8])
    tb_predictions.append(tb_prediction)
    tb_reasonings.append(tb_reasoning)
    pb_predictions.append(pb_prediction)
    pb_reasonings.append(pb_reasoning)

df['TBe_prediction'] = tb_predictions
df['Tbe_reasoning'] = tb_reasonings
df['PBu_prediction'] = pb_predictions
df['PBu_reasoning'] = pb_reasonings

df.to_csv("IRF_var_GPT4.csv", index=False)
csv_file = "IRF_var_GPT4_bad_sample_index.csv"

# Writing the list of tuples to a CSV file

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Writing the header (if necessary)
    writer.writerow(["Row_num", "Error_msg"])
    
    # Writing the data
    writer.writerows(bad_sample)

print(f"Data saved to {csv_file}")
'''

In [ ]:
# MultiWD
'''
df = pd.read_csv("MultiWD_var.csv")
system_prompt = "You are a mental health specialist. Analyze the provided text for signs of mental health issues like depression, anxiety, or other mental disorders. Take social factors such as race, gender, age, sexuality, nationality, and religion into account. Provide a brief explanation of your prediction considering both social factors and the text given."

bad_sample = []

# Initialize predictions and reasonings dictionaries outside the loop
dimensions = ["Spiritual", "Physical", "Intellectual", "Social", "Vocational", "Emotional"]
predictions = {dimension: [] for dimension in dimensions}
reasonings = {dimension: [] for dimension in dimensions}

for i, row in df.iterrows():
    user_prompt = f"Given the post: '{row['text']}', analyze the content to identify relevant wellness dimensions. The output should be in the following format:\n\n'Spiritual: OUTPUT: 0 (No) or 1 (Yes); REASONING:'\n\n'Physical: OUTPUT: 0 (No) or 1 (Yes); REASONING:\n\n'Intellectual: OUTPUT: 0 (No) or 1 (Yes); REASONING:'\n\n'Social: OUTPUT: 0 (No) or 1 (Yes); REASONING:\n\n'Vocational: OUTPUT: 0 (No) or 1 (Yes); REASONING:'\n\n'Emotional: OUTPUT: 0 (No) or 1 (Yes); REASONING:'"
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        content = response.choices[0].message.content
        time.sleep(0.1)
        
        for dimension in dimensions:
            # Use regex to find the dimension with or without space
            dim_match = re.search(f"{dimension}: ?OUTPUT:", content)
            if dim_match:
                dim_start = dim_match.start()
                dim_segment = content[dim_start:].split("\n", 1)[0].strip()
                if "OUTPUT: 1" in dim_segment:
                    predictions[dimension].append(1)
                elif "OUTPUT: 0" in dim_segment:
                    predictions[dimension].append(0)
                else:
                    predictions[dimension].append(None)

                reasoning_start = dim_segment.find("REASONING:")
                if reasoning_start != -1:
                    reasoning = dim_segment[reasoning_start + len("REASONING:"):].strip()
                else:
                    reasoning = ""

                reasonings[dimension].append(reasoning)
            else:
                predictions[dimension].append(None)
                reasonings[dimension].append("")
        
        if i % 50 == 0:
            print('Progress:', i, 'finished')
    
    except Exception as e:
        # Set current predictions and reasonings to None
        for dimension in dimensions:
            predictions[dimension].append(None)
            reasonings[dimension].append(None)
        bad_sample.append((i, str(e)))
        print('Error:', i, str(e)[:8])

# Assign predictions and reasonings to dataframe columns
for dimension in dimensions:
    df[f'{dimension}_prediction'] = predictions[dimension]
    df[f'{dimension}_reasoning'] = reasonings[dimension]

df.to_csv("MultiWD_var_GPT4.csv", index=False)
csv_file = "MultiWD_var_GPT4_bad_sample_index.csv"

# Writing the list of tuples to a CSV file

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Writing the header (if necessary)
    writer.writerow(["Row_num", "Error_msg"])
    
    # Writing the data
    writer.writerows(bad_sample)

print(f"Data saved to {csv_file}")
'''

In [ ]:
# SAD
'''
df = pd.read_csv("SAD.csv")
system_prompt = "You are a mental health specialist. Analyze the provided text for signs of mental health issues like depression, anxiety, or other mental disorders. Take social factors such as race, gender, age, sexuality, nationality, and religion into account. Provide a brief explanation of your prediction considering both social factors and the text given."

bad_sample = []

dimensions = ["Financial_Problem", "Everyday_Decision_Making", "Emotional_Turmoil", "School", "Family_Issues", "Social_Relationships", "Work", "Health_Fatigue_Physical_Pain", "Other"]
predictions = {dimension: [] for dimension in dimensions}
reasonings = {dimension: [] for dimension in dimensions}

for i, row in df.iterrows():
    user_prompt = f"Given the post: '{row['text']}', analyze the content to identify all relevant stressor categories. Provide predictions for each of the following categories. The output should be in the following format:\n\n'Financial_Problem: 0 (No) or 1 (Yes); REASONING:'\n\n'Everyday_Decision_Making: 0 (No) or 1 (Yes); REASONING:'\n\n'Emotional_Turmoil: 0 (No) or 1 (Yes); REASONING:'\n\n'School: 0 (No) or 1 (Yes); REASONING:'\n\n'Family Issues: 0 (No) or 1 (Yes); REASONING:'\n\n'Social_Relationships: 0 (No) or 1 (Yes); REASONING:'\n\n'Work: 0 (No) or 1 (Yes); REASONING:'\n\n'Health_Fatigue_Physical_Pain: 0 (No) or 1 (Yes); REASONING:'\n\n'Other: 0 (No) or 1 (Yes); REASONING:'"
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        content = response.choices[0].message.content
        time.sleep(0.1)
        
        for dimension in dimensions:
            dim_match = re.search(f"{dimension}: (\d) \(\w+\); REASONING: (.*?)(?=(?:'{dimensions[-1]}:|$))", content, re.DOTALL)
            if dim_match:
                output = int(dim_match.group(1))
                reasoning = dim_match.group(2).strip()
                predictions[dimension].append(output)
                reasonings[dimension].append(reasoning)
            else:
                predictions[dimension].append(None)
                reasonings[dimension].append("")
        
        if i % 50 == 0:
            print('Progress:', i, 'finished')
    
    except Exception as e:
        # Set current predictions and reasonings to None
        for dimension in dimensions:
            predictions[dimension].append(None)
            reasonings[dimension].append(None)
        bad_sample.append((i, str(e)))
        print('Error:', i, str(e)[:8])

# Assign predictions and reasonings to dataframe columns
for dimension in dimensions:
    df[f'{dimension}_prediction'] = predictions[dimension]
    df[f'{dimension}_reasoning'] = reasonings[dimension]

df.to_csv("SAD_var_GPT4.csv", index=False)
csv_file = "SAD_var_GPT4_bad_sample_index.csv"

# Writing the list of tuples to a CSV file

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Writing the header (if necessary)
    writer.writerow(["Row_num", "Error_msg"])
    
    # Writing the data
    writer.writerows(bad_sample)

print(f"Data saved to {csv_file}")
'''